# Audio Lab for PYNQ

## Import the overlay

In [ ]:
import audio_lab_pynq as aud
ol = aud.AudioLabOverlay()

## Play with the ADAU1761 CODEC

We expose a Python-based driver for the ADAU1761 audio CODEC, with named registers. Have a look at the format of each register [here](https://www.analog.com/media/en/technical-documentation/data-sheets/ADAU1761.pdf) and play with some of the settings at run-time.

In [ ]:
ol.codec.R30_PLAYBACK_HEADPHONE_RIGHT_VOLUME_CONTROL = 0xE7

## Configurable routing

We've littered this design with AXI-Stream switches meaning that we can, in real-time, swap between different audio sources, effects, and sinks.

So far there are two of each:

  * _Sources_: Line-in jack or DMA
  * _Effects_: Passthrough or low pass filter ([Clash](clash-lang.org))
  * _Sinks_: Headphone jack or DMA
  
Have a play with these. Just now you can only pick a single path through these switches, but the hardware should support multiple(?). Maybe a special DSL would be nice here.

In [ ]:
ol.route(
    aud.XbarSource.line_in,
    aud.XbarEffect.passthrough,
    aud.XbarSink.dma
)

## Using DMA sinks/sources

You can use recorded data as a source, or record data as a sink. Make sure you update the `.route` accordingly.

First, make a physically contiguous buffer.

In [ ]:
from pynq import Xlnk
import numpy as np
xlnk = Xlnk()

dma_array = xlnk.cma_array(shape=(48000, 2), dtype=np.int32)

Now we could use it as a source...

In [ ]:
ol.axi_dma_0.sendchannel.transfer(dma_array)
ol.axi_dma_0.sendchannel.wait()

Or a sink.

In [ ]:
ol.axi_dma_0.recvchannel.transfer(dma_array)
ol.axi_dma_0.recvchannel.wait()

## Plotting waveforms

We can use something like plotly express to get nice visualisations of any captured signals... perhaps to do some analysis of the effects we're applying.

In [ ]:
import plotly.express as px
import pandas as pd

fs = 48000
n  = 48000
dataset = pd.DataFrame({
    'time' : np.array(range(n))/fs,
    'left' : dma_array[:, 0],
    'right': dma_array[:, 1]
})

# Filter zero samples at end of buffer
dataset = dataset[
    (dataset['left']!=0)&
    (dataset['right']!=0)
]

# Plot
df_melt = dataset.melt(id_vars='time', value_vars=['left', 'right'])
px.line(df_melt, x='time' , y='value' , color='variable')